In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
import random

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ALL_MATCHES = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')
all_players = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_players.csv')

<ipython-input-3-4c62233f470e>:1: DtypeWarning: Columns (9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  ALL_MATCHES = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tennis Project/all_matches.csv')


In [ ]:
# Table of features for model
# Engineer feature by feature below

# Drop unknown surface rows
ALL_MATCHES = ALL_MATCHES[ALL_MATCHES['surface'] != 'unknown']

# Drop unknown ranking rows
ALL_MATCHES = ALL_MATCHES[ALL_MATCHES['winner_rank'].isnull() == False]
ALL_MATCHES = ALL_MATCHES[ALL_MATCHES['loser_rank'].isnull() == False]
ALL_MATCHES = ALL_MATCHES.reset_index(drop=True)

features = ALL_MATCHES[['surface', 'winner_rank', 'loser_rank']]

In [ ]:
features

,surface,winner_rank,loser_rank
0,grass,3.0,23.0
1,grass,22.0,30.0
2,grass,55.0,58.0
3,grass,15.0,39.0
4,grass,18.0,97.0
...,...,...,...
144634,clay,7.0,56.0
144635,clay,29.0,47.0
144636,clay,117.0,19.0
144637,clay,7.0,44.0


In [ ]:
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65019 entries, 0 to 65018
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_id    65019 non-null  int64  
 1   hand         65002 non-null  object 
 2   dob          46447 non-null  float64
 3   ioc          64343 non-null  object 
 4   height       2819 non-null   float64
 5   wikidata_id  4894 non-null   object 
 6   name         64134 non-null  object 
 7   hard_elo     65019 non-null  float64
 8   clay_elo     65019 non-null  float64
 9   grass_elo    65019 non-null  float64
 10  carpet_elo   65019 non-null  float64
 11  overall_elo  65019 non-null  float64
dtypes: float64(7), int64(1), object(4)
memory usage: 6.0+ MB


In [ ]:
# Surface Elo, Elo_surface_overall_avg

features['winner_elo_surface'] = 0
features['loser_elo_surface'] = 0
features['winner_elo'] = ALL_MATCHES['winner_elo_before']
features['loser_elo'] = ALL_MATCHES['loser_elo_before']

for i in range(len(ALL_MATCHES)):
    surface = ALL_MATCHES.loc[i, 'surface']
    features.at[i, 'winner_elo_surface'] = ALL_MATCHES.loc[i, f'winner_{surface}_elo_before']
    features.at[i, 'loser_elo_surface'] = ALL_MATCHES.loc[i, f'loser_{surface}_elo_before']

features['winner_avged_elo'] = (features['winner_elo_surface'] + features['winner_elo']) / 2
features['loser_avged_elo'] = (features['loser_elo_surface'] + features['loser_elo']) / 2

<ipython-input-5-309a8245bf48>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['winner_elo_surface'] = 0
<ipython-input-5-309a8245bf48>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['loser_elo_surface'] = 0
<ipython-input-5-309a8245bf48>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
# h2h functions

def h2h(p1_id, p2_id, i):
    """
    Returns p1 wins, p2 wins prior to match
    """
    df = ALL_MATCHES.iloc[:i]
    p1_wins = len(df[(df['winner_id'] == p1_id) & (df['loser_id'] == p2_id)])
    p2_wins = len(df[(df['winner_id'] == p2_id) & (df['loser_id'] == p1_id)])

    return p1_wins, p2_wins


def surface_h2h(p1_id, p2_id, surface, i):
    """
    Returns surface p1 wins, p2 wins prior to match
    """
    df = ALL_MATCHES.iloc[:i]
    p1_wins = len(df[(df['winner_id'] == p1_id) & (df['loser_id'] == p2_id) & (df['surface'] == surface)])
    p2_wins = len(df[(df['winner_id'] == p2_id) & (df['loser_id'] == p1_id) & (df['surface'] == surface)])

    return p1_wins, p2_wins

In [ ]:
# Add h2h

features['winner_wins'] = 0
features['loser_wins'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(i/len(ALL_MATCHES) * 100, ' %')
    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    winner_wins, loser_wins = h2h(winner_id, loser_id, i)
    features.at[i, 'winner_wins'] = winner_wins
    features.at[i, 'loser_wins'] = loser_wins

99.55821044116732  %


In [ ]:
# Add surface h2h

features['winner_wins_surface'] = 0
features['loser_wins_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')
    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    surface = ALL_MATCHES.loc[i, 'surface']
    winner_wins, loser_wins = surface_h2h(winner_id, loser_id, surface, i)
    features.at[i, 'winner_wins_surface'] = winner_wins
    features.at[i, 'loser_wins_surface'] = loser_wins

99.6  %


In [ ]:
# Surface form, season form

def surface_form(player_id, surface, date):
    """
    Returns win % over last 12 months on a given surface.
    Date format: YYYYMMDD.
    """
    df = ALL_MATCHES
    year_ago = date - 10000
    num_wins = len(df[(df['winner_id'] == player_id) & (df['surface'] == surface) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])
    num_losses = len(df[(df['loser_id'] == player_id) & (df['surface'] == surface) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])

    return num_wins / (num_wins + num_losses)


def season_form(player_id, date):
    """
    Returns win % over last 12 months.
    Date format: YYYYMMDD.
    """
    df = ALL_MATCHES
    year_ago = date - 10000
    num_wins = len(df[(df['winner_id'] == player_id) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])
    num_losses = len(df[(df['loser_id'] == player_id) & (df['tourney_date'] > year_ago) & (df['tourney_date'] <= date)])

    return num_wins / (num_wins + num_losses)

In [ ]:
# Add season and surface form features

features['winner_12month_form'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']

    winner_form = season_form(winner_id, tourney_date)
    features.at[i, 'winner_12month_form'] = winner_form

99.6  %


In [ ]:
features['loser_12month_form'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']

    loser_form = season_form(loser_id, tourney_date)
    features.at[i, 'loser_12month_form'] = loser_form

99.6  %


In [ ]:
features['winner_12month_form_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    winner_id = ALL_MATCHES.loc[i, 'winner_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    winner_form_surface = surface_form(winner_id, surface, tourney_date)
    features.at[i, 'winner_12month_form_surface'] = winner_form_surface

99.6  %


In [ ]:
features['loser_12month_form_surface'] = 0

for i in range(len(ALL_MATCHES)):
    if i % 1000 == 0:
        clear_output()
        print(np.round(i/len(ALL_MATCHES) * 100, 1), ' %')

    loser_id = ALL_MATCHES.loc[i, 'loser_id']
    tourney_date = ALL_MATCHES.loc[i, 'tourney_date']
    surface = ALL_MATCHES.loc[i, 'surface']

    loser_form_surface = surface_form(loser_id, surface, tourney_date)
    features.at[i, 'loser_12month_form_surface'] = loser_form_surface

99.6  %


In [ ]:
features['winner_hand'] = ALL_MATCHES['winner_hand'].map({'R': 0, 'L': 1})
features['loser_hand'] = ALL_MATCHES['loser_hand'].map({'R': 0, 'L': 1})

In [ ]:
names_dict = {'winner_rank': 'A_rank', 'loser_rank': 'B_rank',
              'winner_elo_surface': 'A_elo_surface', 'loser_elo_surface': 'B_elo_surface',
              'winner_elo': 'A_elo', 'loser_elo': 'B_elo',
              'winner_avged_elo': 'A_avged_elo', 'loser_avged_elo': 'B_avged_elo',
              'winner_wins': 'A_wins', 'loser_wins': 'B_wins',
              'winner_wins_surface': 'A_wins_surface', 'loser_wins_surface': 'B_wins_surface',
              'winner_12month_form': 'A_12month_form', 'loser_12month_form': 'B_12month_form',
              'winner_12month_form_surface': 'A_12month_form_surface', 'loser_12month_form_surface': 'B_12month_form_surface',
              'surface': 'surface', 'winner_hand': 'A_hand', 'loser_hand': 'B_hand'}

features = features.rename(columns=names_dict)

In [ ]:
from google.colab import files
features.to_csv('features_processed.csv', index=False)
files.download('features_processed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
### Possible additions:
# Recent form (e.g. last 3 months)
# Recovering from injury -- T/F
# Performances in same tournament level
# Highest previous round in tournament
# Seeding (if available)

def injury(player_id, date):
    """
    Returns True if <=10 matches in last 12 months; False otherwise.
    """
    df = ALL_MATCHES
    year_ago = date - 10000
    num_matches = len(df[(df['winner_id'] == player_id) & (df['tourney_date'] > year_ago)] & (df['tourney_date'] < date))

    return num_matches <= 10
